Machine Learning Feature Extraction Optimization Using Evolutionary Computation Algorithms


Problem Definition

Dataset Explaination

Strategies


**Possible datasets**

[Heart Failure Prediction Dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data) (13 features)

[Real or Fake Jobposting Prediction Dataset](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction) (18 features)

[Footballer Price Prediction Dataset](https://www.kaggle.com/datasets/rashadrmammadov/football-teams-price-prediction) (26 features)

[Loan Default Prediction Dataset](https://www.kaggle.com/datasets/hemanthsai7/loandefault) (35 features)

[Bankruptcy Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction) (96 features)


In [7]:
import pandas as pd
import numpy as np

Random Forest

XGBoost

Logistic Regression

Linear Regression

FCNN(?)

SVM(?)

In [ ]:
!pip install kagglehub

In [4]:
import kagglehub


path_bankrupt = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")
path_loan = kagglehub.dataset_download("hemanthsai7/loandefault")
path_footballer = kagglehub.dataset_download("rashadrmammadov/football-teams-price-prediction")
path_job = kagglehub.dataset_download("shivamb/real-or-fake-fake-jobposting-prediction")
path_heart = kagglehub.dataset_download("andrewmvd/heart-failure-clinical-data")


/home/tugberk/.cache/kagglehub/datasets/fedesoriano/company-bankruptcy-prediction/versions/2


In [5]:
heart_data_path = f"{path_heart}/heart_failure_clinical_records_dataset.csv"
job_data_path = f"{path_job}/fake_job_postings.csv"
footballer_data_path = f"{path_footballer}/Football_teams_price_data.csv"
loan_data_path = f"{path_loan}/train.csv"
bankrupt_data_path = f"{path_bankrupt}/data.csv"


In [10]:
heart_data = pd.read_csv(heart_data_path)
job_data = pd.read_csv(job_data_path)
footballer_data = pd.read_csv(footballer_data_path)
loan_data = pd.read_csv(loan_data_path)
bankrupt_data = pd.read_csv(bankrupt_data_path)

13

In [ ]:
#Algorithms : GA, DE, ES
#Datasets: heart_data(13 dimension), job_data(18 dimension), footballer_data(26 dimension) 
#          loan_data(35 dimension), bankrupt_data(96 dimension)
def evolution(algorithm, dataset, pop_size, gens = 100, mut_rate = 0.1, cross_rate = 0.9):

    dimension = len(dataset.columns)
    population_size = pop_size
    generations = gens
    mutation_rate = mut_rate
    crossover_rate = cross_rate

    if algorithm == "GA":
        genetic_algorithm(dataset, dimension, population_size, generations, mutation_rate, crossover_rate)
    elif algorithm == "DE":
        differential_evolution(dataset, dimension, population_size, generations, mutation_rate)
    elif algorithm == "ES":
        evolution_strategy(dataset, dimension, population_size, generations, mutation_rate)